# Core

> Core functions

In [ ]:
#| default_exp layers.logit

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastai.vision.all import torch, nn, np

In [ ]:
#| export
class ConditionalLinear(nn.Module):
    def __init__(self, device='cuda', name='linear_transformation'):
        super(ConditionalLinear, self).__init__()
        self.name = name

        self.iso_vals = torch.tensor([100, 400, 800, 1600, 3200], dtype=torch.float32, device=device)
        self.cam_vals = torch.tensor([0, 1, 2, 3, 4], dtype=torch.float32, device=device)  # 'IP', 'GP', 'S6', 'N6', 'G4'

        self.log_scale = nn.Parameter(torch.zeros(25), requires_grad=True)
        self.bias = nn.Parameter(torch.zeros(25), requires_grad=True)

    def _inverse(self, z, **kwargs):
        gain_one_hot = self.iso_vals == torch.mean(kwargs['iso'], dim=[1, 2, 3]).unsqueeze(1)
        iso = gain_one_hot.nonzero()[:, 1]
        cam_one_hot = self.cam_vals == torch.mean(kwargs['cam'], dim=[1, 2, 3]).unsqueeze(1)
        cam = cam_one_hot.nonzero()[:, 1]
        iso_cam = iso * 5 + cam
        iso_cam = torch.arange(0, 25).cuda() == iso_cam.unsqueeze(1)

        log_scale = self.log_scale.unsqueeze(0).repeat_interleave(z.shape[0], dim=0)[iso_cam]
        bias = self.bias.unsqueeze(0).repeat_interleave(z.shape[0], dim=0)[iso_cam]

        x = (z - bias.reshape((-1, 1, 1, 1))) / torch.exp(log_scale.reshape((-1, 1, 1, 1)))
        return x

    def _forward_and_log_det_jacobian(self, x, **kwargs):
        gain_one_hot = self.iso_vals == torch.mean(kwargs['iso'], dim=[1, 2, 3]).unsqueeze(1)
        iso = gain_one_hot.nonzero()[:, 1]
        cam_one_hot = self.cam_vals == torch.mean(kwargs['cam'], dim=[1, 2, 3]).unsqueeze(1)
        cam = cam_one_hot.nonzero()[:, 1]
        iso_cam = iso * 5 + cam
        iso_cam = torch.arange(0, 25).cuda() == iso_cam.unsqueeze(1)

        log_scale = self.log_scale.unsqueeze(0).repeat_interleave(x.shape[0], dim=0)[iso_cam]
        bias = self.bias.unsqueeze(0).repeat_interleave(x.shape[0], dim=0)[iso_cam]
        
        z = x * torch.exp(log_scale.reshape((-1, 1, 1, 1))) + bias.reshape((-1, 1, 1, 1))
        log_abs_det_J_inv = log_scale * np.prod(x.shape[1:])

        return z, log_abs_det_J_inv


In [ ]:
#| export
class ConditionalLinearExp2(nn.Module):
    def __init__(self, device='cuda', name='linear_transformation_exp2'):
        super(ConditionalLinearExp2, self).__init__()
        self.name = name

        self.iso_vals = torch.tensor([100, 400, 800, 1600, 3200], dtype=torch.float32, device=device)
        self.cam_vals = torch.tensor([0, 1, 2, 3, 4], dtype=torch.float32, device=device)  # 'IP', 'GP', 'S6', 'N6', 'G4'

        self.log_scale = nn.Parameter(torch.zeros(25, 3), requires_grad=True)
        self.bias = nn.Parameter(torch.zeros(25, 3), requires_grad=True)

    def _inverse(self, z, **kwargs):
        gain_one_hot = self.iso_vals == torch.mean(kwargs['iso'], dim=[1, 2, 3]).unsqueeze(1)
        iso = gain_one_hot.nonzero()[:, 1]
        cam_one_hot = self.cam_vals == torch.mean(kwargs['cam'], dim=[1, 2, 3]).unsqueeze(1)
        cam = cam_one_hot.nonzero()[:, 1]
        iso_cam = iso * 5 + cam
        iso_cam = torch.arange(0, 25).cuda() == iso_cam.unsqueeze(1)

        log_scale = self.log_scale.unsqueeze(0).repeat_interleave(z.shape[0], dim=0)[iso_cam]
        bias = self.bias.unsqueeze(0).repeat_interleave(z.shape[0], dim=0)[iso_cam]

        x = (z - bias.reshape((-1, z.shape[1], 1, 1))) / torch.exp(log_scale.reshape((-1, z.shape[1], 1, 1)))
        return x
    def _forward_and_log_det_jacobian(self, x, **kwargs):
        gain_one_hot = self.iso_vals == torch.mean(kwargs['iso'], dim=[1, 2, 3]).unsqueeze(1)
        iso = gain_one_hot.nonzero()[:, 1]
        cam_one_hot = self.cam_vals == torch.mean(kwargs['cam'], dim=[1, 2, 3]).unsqueeze(1)
        cam = cam_one_hot.nonzero()[:, 1]
        iso_cam = iso * 5 + cam
        iso_cam = torch.arange(0, 25).cuda() == iso_cam.unsqueeze(1)

        log_scale = self.log_scale.unsqueeze(0).repeat_interleave(x.shape[0], dim=0)[iso_cam]
        bias = self.bias.unsqueeze(0).repeat_interleave(x.shape[0], dim=0)[iso_cam]
        z = x * torch.exp(log_scale.reshape((-1, x.shape[1], 1, 1))) + bias.reshape((-1, x.shape[1], 1, 1))
        log_abs_det_J_inv = torch.sum(log_scale * np.prod(x.shape[2:]), dim=1)

        return z, log_abs_det_J_inv


In [ ]:
#| export
class Gamma(nn.Module):
    def __init__(self, device='cuda', name='gamma'):
        super(Gamma, self).__init__()
        self.name = name
        self.gamma = nn.Parameter(torch.tensor(2.2), requires_grad=True)
        self.constant = 0.00005

    def _inverse(self, z, **kwargs):
        z = z.clamp(min=0)
        # print(torch.min(z), torch.max(z))
        x = z**(1/self.gamma)
        x -= self.constant
        return x

    def _forward_and_log_det_jacobian(self, x, **kwargs):
        x += self.constant
        z = x**self.gamma
        log_abs_det_J_inv = torch.sum(torch.log(self.gamma) + (self.gamma-1)*torch.log(x), dim=[1, 2, 3])

        if 'writer' in kwargs.keys():
            kwargs['writer'].add_scalar('model/' + self.name + '_train', self.gamma, kwargs['step'])

        return z, log_abs_det_J_inv

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()